In [32]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
from spellchecker import SpellChecker
import joblib

In [33]:
def encode_label(string):
    if(string=="ham"):
        return 0
    else:
        return 1

In [34]:
data=pd.read_csv("./DataSet/spam.csv",encoding="ISO-8859-1")
data["v1"]=data["v1"].apply(encode_label)
result=data["v1"]
data=data.drop(["v1","Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)

In [35]:
new_data=pd.read_csv("./DataSet/spam_ham_dataset.csv")
new_data.rename(columns = {'text':'v2',"label_num":"v1"}, inplace = True)
new_resut=new_data["v1"]
new_data=new_data.drop(["label","v1","Unnamed: 0"],axis=1)

In [36]:
new_custom_email=pd.read_csv("./DataSet/Custom_Email.csv")
new_custom_email_result=new_custom_email["v1"]
new_custom_email=new_custom_email.drop(["v1","Unnamed: 0"],axis=1)
new_custom_email

,v2
0,Internshala Trainings Gain an edge in the job ...
1,Ready to deepen your AI skills and transform y...
2,"Zakie, earn high stipend in your preferred fie..."
3,Get 3+1 months of Zomato Gold at just ₹20 and ...
4,[ Application deadline has been extended! -...
...,...
275,[ and build your research projects - Courser...
276,"Dads, if you’re reading this please know that ..."
277,Loading the image... Registered already? Be pu...
278,Internshala Trainings FLAT 55% OFF JUST FOR YO...


In [37]:
new_custom_email_result

0      1
1      1
2      1
3      1
4      1
      ..
275    1
276    1
277    1
278    1
279    1
Name: v1, Length: 280, dtype: int64

In [38]:
data=pd.concat([data,new_data],axis=0)
result=pd.concat([result,new_resut],axis=0)
data=pd.concat([data,new_custom_email],axis=0)
result=pd.concat([result,new_custom_email_result],axis=0)

In [39]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [40]:
def html_tag_removal(text):
    html_tags_pattern = r'<.*?>'
    text_without_html_tags = re.sub(html_tags_pattern, '', text)
    return text_without_html_tags

In [41]:
stemmer = PorterStemmer()
def stem_words(words):
    return [stemmer.stem(word) for word in words]

In [42]:


data["v2"]=[row.lower() for row in data["v2"]]
data["v2"]=data["v2"].apply(remove_urls)
data["v2"]=data["v2"].apply(html_tag_removal)
data["v2"] = data["v2"].replace(to_replace=r'[^\w\s]', value='', regex=True)
data["v2"] = [emoji.demojize(doc) for doc in data["v2"]]


In [43]:
data

,v2
0,go until jurong point crazy available only in ...
1,ok lar joking wif u oni
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor u c already then say
4,nah i dont think he goes to usf he lives aroun...
...,...
275,and build your research projects coursera ...
276,dads if youre reading this please know that th...
277,loading the image registered already be punctu...
278,internshala trainings flat 55 off just for you...


In [44]:
data["v2"]

0      go until jurong point crazy available only in ...
1                                ok lar joking wif u oni
2      free entry in 2 a wkly comp to win fa cup fina...
3            u dun say so early hor u c already then say
4      nah i dont think he goes to usf he lives aroun...
                             ...                        
275      and build your research projects   coursera ...
276    dads if youre reading this please know that th...
277    loading the image registered already be punctu...
278    internshala trainings flat 55 off just for you...
279    zakie earn high stipend in your preferred fiel...
Name: v2, Length: 11023, dtype: object

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
data["v2"]=[str(row).split(" ") for row in data["v2"]]
data['v2'] = data['v2'].apply(lambda x: ', '.join(map(str, x)))
pd.DataFrame({"v2":data["v2"],"v1":result}).to_csv("final.csv",index=False)
tfidf_vectorizer = TfidfVectorizer()
array=tfidf_vectorizer.fit_transform(data["v2"])
data=array.toarray()
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.under_sampling import RandomUnderSampler
enn = RandomUnderSampler()

data,result=enn.fit_resample(data,result)


In [47]:
x_train,x_test,y_train,y_test=train_test_split(data,result,test_size=0.2)

In [48]:
data.shape

(5052, 56723)

In [49]:
{}

{}

In [50]:
result.value_counts()

v1
0    2526
1    2526
Name: count, dtype: int64

In [51]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
gNB=RandomForestClassifier()
# scores = cross_val_score(gNB, data, result, cv=10)
gNB.fit(x_train,y_train)

RandomForestClassifier()

In [52]:
# scores.mean()

In [53]:
pred=gNB.predict(x_test)
pred

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

In [54]:
from sklearn.metrics import  classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       493
           1       0.97      0.92      0.95       518

    accuracy                           0.95      1011
   macro avg       0.95      0.95      0.95      1011
weighted avg       0.95      0.95      0.95      1011



In [55]:
# export = LazyExport(gNB)
# export.save("./Models/spam_detector.json", indent=4)

In [56]:
y_test.value_counts()

v1
1    518
0    493
Name: count, dtype: int64

In [57]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       493
           1       0.97      0.92      0.95       518

    accuracy                           0.95      1011
   macro avg       0.95      0.95      0.95      1011
weighted avg       0.95      0.95      0.95      1011



In [58]:
input_text="""
Hi Zakie,How Are You?
"""
def input_preprocess(txt):
    txt=txt.lower()
    txt=remove_urls(txt)
    txt=html_tag_removal(txt)
    # print(txt)
    txt = re.sub(r'[^\w\s]','',txt)
    
    array=tfidf_vectorizer.transform([txt])
    print(array.shape)
    return array.toarray()

In [59]:
(input_preprocess(input_text))

(1, 56723)


array([[0., 0., 0., ..., 0., 0., 0.]])

In [60]:
# qddl jsqs bdwo theq

In [61]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred)

In [62]:

# joblib.dump(gNB, 'Spam_Model.pkl') 